### Twitter Archive to CSV Converter

In [30]:
import os                           # For listing directories
import json                         # For JSON-structured file reader
from operator import itemgetter     # For sorting
import csv                          # For loading to target CSV

# Path to the tweet archives folder (data)
path = '..//..//..//..//tweets'

# Container for the end result
tweets = []

# Walk through the archived files
for(root, dirs, files) in os.walk(path):
    # Iterate the files
    for file in files:
        # Open each file
        with open (f'{path}//{file}') as f:
            # Skip the first line (variable assignment)
            next(f)
            for tweet in json.load(f):
                # Create row variable
                temp_tweet = []

                # Tweet ID
                temp_tweet.append(tweet["id_str"])
                # Tweet Link
                temp_tweet.append(f'https://twitter.com/{tweet["user"]["screen_name"]}/status/{tweet["id_str"]}')
                # Tweet ID
                temp_tweet.append(tweet["created_at"])
                # Tweet Content
                temp_tweet.append(tweet["text"])

                # Completed, add to master list
                tweets.append(temp_tweet)

# Redo the order since content of data files are unordered
tweets = sorted(tweets, key=itemgetter(0))

# Write into the target CSV
with open('Tweet Collection.csv', 'w', newline='',encoding='utf-8') as file:
    # Add proper header
    csv_header = csv.DictWriter(file, delimiter=',', fieldnames=['Tweet ID', 'Tweet Link', 'Date','Tweet Content'])
    csv_header.writeheader()

    # Write file
    csv_writer = csv.writer(file)
    for i in tweets:
        csv_writer.writerow(i)

print('Done!')

Done!
